In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from denn import *
from fastai.basics import *

In [2]:
D = 2
def fitness_func(indiv, b, t): return (indiv.data**2).sum()
def constraint_func(indiv, b, t): return -b[t] + sum((1/np.sqrt(D))*indiv.data)
ab = [1.] * 100
population = Population.new_random(n=30, dimension=D)

opt = Optimization(population, fitness_func, constraint_func, constraint_params=[ab],
                   max_times=100, frequency=0.1, callbacks=Logger, path='test_logger')
opt.state_dict

{'gen': 0, 'evals': 0, 'time': 0, 'time_evals': 0, 'best': None}

In [3]:
opt.run(200)

Total time: 00:01


In [4]:
opt.logger

Logger(path='test_logger/logger.json')

In [5]:
!cp {opt.logger.path} /home/renato/github/show_evolution/src/data/ea_data.json